REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [1]:
import pandas as pd

In [2]:
bus = pd.read_csv('HW3_Data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('HW3_Data/inspections.csv',dtype={"date": str})

In [3]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [4]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [5]:
total_businesses = len(bus)
print("The number of records in the business data is:", total_businesses)

The number of records in the business data is: 6315


#### Question 2b: How many unique business IDs are there?  

In [6]:
unique_bus_ids = bus['business_id'].nunique()
print("The number of unique business IDs in the business data is:", unique_bus_ids)

The number of unique business IDs in the business data is: 6315


#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [7]:
most_common_bus_names = bus['name'].value_counts().head(5)
print("The 5 most common businesses by name in the business data are presented below with their respective quantities:")
print(most_common_bus_names)

The 5 most common businesses by name in the business data are presented below with their respective quantities:
STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: name, dtype: int64


## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [8]:
postal_code_type = type(bus.loc[1, 'postal_code'])
print("Postal code values are stored as type:", postal_code_type)

Postal code values are stored as type: <class 'str'>


#### Question 3b

What are the unique values of postal_code?

In [9]:
bus['postal_code'].unique()

array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [10]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [11]:
bus_valid = bus[bus['postal_code'].isin(validZip)]
print("The number of businesses whose postal codes show up in the list of valid zipcodes is:", len(bus_valid))

The number of businesses whose postal codes show up in the list of valid zipcodes is: 5999


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [12]:
print("The number of businesses that are missing longitude values, out of the businesses that have postal codes in the list \nof valid zip codes, is:", len(bus_valid[pd.isnull(bus_valid['longitude'])]))

The number of businesses that are missing longitude values, out of the businesses that have postal codes in the list 
of valid zip codes, is: 2483


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [13]:
null_lon_bus = bus_valid[pd.isnull(bus_valid['longitude'])]
null_lon = []
for i in validZip:
    count = 0
    for index, row in null_lon_bus.iterrows():
        if row['postal_code'] == i:
            count += 1
    null_lon.append(count)

not_null_lon_bus = bus_valid[pd.notnull(bus_valid['longitude'])]
not_null_lon = []
for i in validZip:
    count = 0
    for index, row in not_null_lon_bus.iterrows():
        if row['postal_code'] == i:
            count += 1
    not_null_lon.append(count)

validZip_summary = pd.DataFrame(list(zip(validZip, null_lon, not_null_lon)), columns =['postal_code', 'null_lon', 'not_null_lon'])
validZip_summary

,postal_code,null_lon,not_null_lon
0,94102,196,262
1,94103,252,306
2,94104,59,74
3,94105,100,126
4,94107,247,204
5,94108,82,142
6,94109,146,237
7,94110,251,334
8,94111,115,167
9,94112,59,131


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [14]:
# Obtaining the proportion of businesses per valid zipcode that are missing longitude
prop_null_lon = []
for i in range(len(validZip)):
    prop_null_lon.append(validZip_summary.loc[i, 'null_lon'] / (validZip_summary.loc[i, 'null_lon'] + validZip_summary.loc[i, 'not_null_lon']))

validZip_summary_prop = pd.DataFrame(list(zip(validZip, null_lon, not_null_lon, prop_null_lon)), columns =['postal_code', 'null_lon', 'not_null_lon', 'prop_null_lon'])
validZip_summary_prop

,postal_code,null_lon,not_null_lon,prop_null_lon
0,94102,196,262,0.427948
1,94103,252,306,0.451613
2,94104,59,74,0.443609
3,94105,100,126,0.442478
4,94107,247,204,0.547672
5,94108,82,142,0.366071
6,94109,146,237,0.381201
7,94110,251,334,0.429060
8,94111,115,167,0.407801
9,94112,59,131,0.310526


In [15]:
# Obtaining the top 5 five zipcodes in descending order of proportion missing longitude
validZip_summary_prop.nlargest(5, ['prop_null_lon'])

,postal_code,null_lon,not_null_lon,prop_null_lon
4,94107,247,204,0.547672
18,94124,88,76,0.536585
21,94132,65,69,0.485075
10,94114,108,122,0.469565
16,94122,123,147,0.455556


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [16]:
# Obtaining the number of records in the inspection data
total_inspections = len(ins)
print("The number of records in the inspection data is:", total_inspections)

The number of records in the inspection data is: 15430


In [17]:
# Obtaining the number of unique businesses there are in the inspection data
unique_bus_ids_ins = ins['business_id'].nunique()
print("The number of unique business IDs in the inspection data is:", unique_bus_ids_ins)
print("and the number of unique business IDs in the business data is:", unique_bus_ids)

The number of unique business IDs in the inspection data is: 5730
and the number of unique business IDs in the business data is: 6315


#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [18]:
# Obtaining the data type of "type" in the inspection data
type_type = type(ins.loc[1, 'type'])
print("Type values are stored as type:", type_type)

Type values are stored as type: <class 'str'>


In [19]:
# Obtaining the types of inspections that occur more than 10 times
ins_type = ins['type'].value_counts().head(5)
print("The types of inspections in the inspection data are presented below with their respective quantities:")
print(ins_type)

validType = ['routine']

The types of inspections in the inspection data are presented below with their respective quantities:
routine      15429
complaint        1
Name: type, dtype: int64


In [20]:
# Creating a new dataframe named ins2
ins2 = ins[ins['type'].isin(validType)]
ins2

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine
...,...,...,...,...
15425,89515,98,20161103,routine
15426,89701,100,20161206,routine
15427,89790,96,20161129,routine
15428,89803,100,20161215,routine


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [21]:
# Double checking that dates were passed through as a strings; I modified the initial code to:
# ins = pd.read_csv('HW3_Data/inspections.csv',dtype={"date": str})
# Since the original code resulted in dates being float type values

date_type = type(ins2.loc[1, 'date'])
print("Date values are stored as type:", date_type)

Date values are stored as type: <class 'str'>


In [23]:
# Adding a new column in ins2 called "year"
ins2['date'] = pd.to_datetime(ins2['date'])
ins2['year'] = ins2['date'].dt.year
ins2.head()

C:\Users\wzwzh\AppData\Local\Temp\ipykernel_17376\1308318655.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ins2['date'] = pd.to_datetime(ins2['date'])
C:\Users\wzwzh\AppData\Local\Temp\ipykernel_17376\1308318655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ins2['year'] = ins2['date'].dt.year


,business_id,score,date,type,year
0,10,82,2016-05-03,routine,2016
1,10,94,2014-07-29,routine,2014
2,10,92,2014-01-14,routine,2014
3,19,94,2016-05-13,routine,2016
4,19,94,2014-11-10,routine,2014


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [24]:
# Obtaining the range of years covered in the ins2 dataframe
oldest_yr = ins2['year'].min()
newest_yr = ins2['year'].max()
print("The range of years covered in the ins2 dataframe is:", oldest_yr, "to", newest_yr)

The range of years covered in the ins2 dataframe is: 2013 to 2016


In [25]:
# Obtaining the number of routine inspections each year
ins_annually = ins2['year'].value_counts().head(5)
print("The occurrence of routine inspections in the inspection data are presented below with their respective quantities per year:")
print(ins_annually)
print("\nEach year has a different number of inspections per year.")

The occurrence of routine inspections in the inspection data are presented below with their respective quantities per year:
2016    5839
2014    5629
2015    3923
2013      38
Name: year, dtype: int64

Each year has a different number of inspections per year.


In [26]:
# Creating a new dataframe named ins3
validYear = [2016, 2015, 2014]
ins3 = ins2[ins2['year'].isin(validYear)]
ins3

,business_id,score,date,type,year
0,10,82,2016-05-03,routine,2016
1,10,94,2014-07-29,routine,2014
2,10,92,2014-01-14,routine,2014
3,19,94,2016-05-13,routine,2016
4,19,94,2014-11-10,routine,2014
...,...,...,...,...,...
15425,89515,98,2016-11-03,routine,2016
15426,89701,100,2016-12-06,routine,2016
15427,89790,96,2016-11-29,routine,2016
15428,89803,100,2016-12-15,routine,2016


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [27]:
target_year = [2016]
ins_2016 = ins3[ins3['year'].isin(target_year)]
ins_2016

,business_id,score,date,type,year
0,10,82,2016-05-03,routine,2016
3,19,94,2016-05-13,routine,2016
6,24,98,2016-10-05,routine,2016
7,24,96,2016-03-11,routine,2016
10,31,84,2016-08-16,routine,2016
...,...,...,...,...,...
15425,89515,98,2016-11-03,routine,2016
15426,89701,100,2016-12-06,routine,2016
15427,89790,96,2016-11-29,routine,2016
15428,89803,100,2016-12-15,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [28]:
# Note: using the original "bus" dataframe instead of the "bus_valid" dataframe created in this assignment to capture all businesses
merged_bus_ins_2016 = pd.merge(bus, ins_2016, on='business_id', how='left')
merged_bus_ins_2016.head(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,2016-05-03,routine,2016.0
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,2016-05-13,routine,2016.0
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,2016-10-05,routine,2016.0
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,2016-03-11,routine,2016.0
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,2016-08-16,routine,2016.0
5,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,84.0,2016-06-14,routine,2016.0
6,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051,78.0,2016-01-04,routine,2016.0
7,48,ART'S CAFE,747 IRVING ST,San Francisco,CA,94122,37.764013,-122.465749,+14156657440,94.0,2016-06-30,routine,2016.0
8,50,SUSHI ZONE,1815 MARKET ST.,San Francisco,CA,94103,37.771437,-122.423892,+14156211114,80.0,2016-06-14,routine,2016.0
9,54,RHODA GOLDMAN PLAZA,2180 POST ST,San Francisco,CA,94115,37.784626,-122.437734,+14153455060,NaN,NaT,NaN,NaN


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [29]:
lowest_scored_bus_2016 = merged_bus_ins_2016.sort_values(by='score').head(20)[['name', 'address', 'score']]
lowest_scored_bus_2016

,name,address,score
4892,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52.0
7324,POKI TIME,2101 LOMBARD ST,55.0
6281,CRAZY PEPPER,2257 SAN JOSE AVE,55.0
4873,UNCLE CAFE,65 WAVERLY PL,55.0
3008,HING WANG BAKERY,339 JUDAH ST,56.0
5627,GOLDEN WOK,295 B ORIZABA AVE,56.0
5383,TOP'S CAFE,2460 SAN BRUNO AVE,57.0
6067,L & G VIETNAMESE SANDWICH,602 EDDY ST,57.0
836,RED A BAKERY,634 CLEMENT ST,57.0
4209,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57.0


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!